# BarrelNet Inference Notebook
This Notebook will run the code to run an inference pipeline for the trained Model, given a single Barrel point cloud sample as input. 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import os
from pathlib import Path

import dill as pickle
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import plotly.graph_objects as go
import visu3d as v3d
import transforms3d as t3d

from burybarrel.transform import icp_translate
from burybarrel.plotting import get_ray_trace
from burybarrel.barrelnet.barrelnet import BarrelNet
from burybarrel.barrelnet.data import pts2inference_format
from burybarrel.synthbarrel import Cylinder, generate_oriented_barrel

In [ ]:
def get_trial_var(trialresults, varname):
    return [trial[varname] for trial in trialresults]

## Loading the occluded dataset (need to use Pointnet generation code)

In [ ]:
with open("../data/synthbarrel/testbarrels_1000_fixed.pkl", "rb") as f:
    synthdict = pickle.load(f)
print(synthdict.keys())

In [ ]:
## Load Model 
model_path = "../checkpoints/pointnet_iter80_fixed.pth"
pointnet = BarrelNet(k=5, normal_channel=False)
pointnet.load_state_dict(torch.load(model_path))
pointnet.cuda().eval()

In [ ]:
# cylnp = random_cylinder_surf([0, 0, 0], [0, 0, height_ratio], 1, 5000).astype(np.float32)
# radius predicted: fraction of height
# normalized space: height is fixed at 1
# height_ratio = 2.5  # height / radius ratio
cylh = 1
ntrials = synthdict["radii"].shape[0]

trialresults = []
# for i in tqdm(range(ntrials)):
for i in tqdm(range(20)):
    results = {}
    cylnp = synthdict["pts"][i].numpy()
    axtruth = synthdict["axis_vectors"][i]
    rtruth = synthdict["radii"][i].numpy()
    # height in generated data is fixed at 1
    yoffsettruth = synthdict["burial_offsets"][i]
    cyltruth = Cylinder.from_axis(axtruth, rtruth, 1, c=[0, yoffsettruth, 0])
    
    results["cyltruth"] = cyltruth
    results["burialtruth"] = cyltruth.get_volume_ratio_monte(5000, planecoeffs=[0, 1, 0, 0])

    axis_pred, r, h, y = pointnet.predict_np(cylnp, height_radius_ratio=1/rtruth)
    
    cylpred = Cylinder.from_axis(axis_pred, r, h, c=[0, y, 0])
    predsurfpts = cylpred.get_random_pts_surf(5000)
    translation = icp_translate(cylnp, predsurfpts, max_iters=5, ntheta=0, nphi=0)
    cylpred = cylpred.translate(-translation)
    
    results["cylpred"] = cylpred
    results["burialpred"] = cylpred.get_volume_ratio_monte(5000, planecoeffs=[0, 1, 0, 0])

    # print("ahAHSFHJKSADHJKFSDHJKDFSHJKFSAD")
    # print(axis_pred, r, h, y)
    # print(axtruth, rtruth, h, yoffsettruth / h)
    
    trialresults.append(results)

    # print("TRUTH")
    # print(f"axis: {cylax}\nradius: {cylr}\nheight: {cylh}\nz-offset: {cylz}")
    # print(f"burial percentage: {burialtruth}")
    # print("PREDICTED")
    # print(radius_pred, zshift_pred, axis_pred)
    # print(f"axis: {axis_pred}\nradius: {r}\nheight: {h}\nz-offset: {z}")
    # print(f"burial percentage: {burialpred}")

    # truthray = v3d.Ray(pos=[0,0,0], dir=cylax)
    # predray = v3d.Ray(pos=[0,0,0], dir=axis_pred)
    # v3d.make_fig([v3d.Point3d(p=cylnp), truthray, predray])
with open("../results/pointnet_synth_results.pkl", "wb") as f:
    pickle.dump(trialresults, f)

In [ ]:
with open("../results/pointnet_synth_results.pkl", "rb") as f:
    trialresults = pickle.load(f)

In [ ]:
i = 15
trialres = trialresults[i]
cyltruth: Cylinder = trialres["cyltruth"]
cylpred: Cylinder = trialres["cylpred"]
truthray = v3d.Ray(pos=cyltruth.c, dir=cyltruth.axis)
predray = v3d.Ray(pos=cylpred.c, dir=cylpred.axis)
fig = v3d.make_fig([v3d.Point3d(p=synthdict["pts"][i].numpy()), truthray, predray])
x, y, z = cyltruth.get_pts_surf()
cylsurftruth = go.Surface(
    x=x, y=y, z=z,
    colorscale="purples",
    #  showscale=False,
    opacity=0.2)
fig.add_trace(cylsurftruth)
x, y, z = cylpred.get_pts_surf()
cylsurfpred = go.Surface(
    x=x, y=y, z=z,
    colorscale="oranges",
    #  showscale=False,
    opacity=0.2)
fig.add_trace(cylsurfpred)
fig.show()
print(cyltruth.get_volume_ratio_monte(10000, planecoeffs=[0, 1, 0, 0]))
print(cylpred.get_volume_ratio_monte(10000, planecoeffs=[0, 1, 0, 0]))

In [ ]:
axtruths = np.array([cyl.axis for cyl in get_trial_var(trialresults, "cyltruth")])
axpreds = np.array([cyl.axis for cyl in get_trial_var(trialresults, "cylpred")])
cossims = np.abs(np.sum(axtruths * axpreds, axis=1))
plt.hist(cossims)
np.mean(cossims)

In [ ]:
np.argmin(cossims)

In [ ]:
np.sum(cossims < 0.9)

In [ ]:
burialerrs = np.abs(np.array(get_trial_var(trialresults, "burialtruth")) - np.array(get_trial_var(trialresults, "burialpred")))
plt.hist(burialerrs)
np.mean(burialerrs)

In [ ]:
i = 2
trialres = trialresults[i]
R = t3d.euler.euler2mat(np.pi/2, 0, 0)
T_yup2zup = np.eye(4)
T_yup2zup[:3, :3] = R
cylnp = synthdict["pts"][i].numpy()
cylnp = (R @ cylnp.T).T
fig = v3d.make_fig([v3d.Point3d(p=cylnp)])
cyltruth: Cylinder = trialres["cyltruth"].transform(T_yup2zup)
# x1truth, x2truth  = get_cyl_endpoints(R @ trialres["axtruth"], 1, trialres["yshifttruth"], axidx=2)
truthray = v3d.Ray(pos=cyltruth.c, dir=cyltruth.axis)
x, y, z = cyltruth.get_pts_surf()
cylsurftruth = go.Surface(
    x=x, y=y, z=z,
    colorscale="oranges",
    surfacecolor=np.zeros_like(x),
    #  showscale=False,
    opacity=0.5)
fig.add_trace(cylsurftruth)
fig.add_trace(get_ray_trace(pos=cyltruth.c, raydir=cyltruth.axis, color="#e81b00", width=5, length=0.7))
# fig.add_traces(get_surface_line_traces(x, y, z, include_horizontal=False, step=5))
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False)
xx, yy = np.meshgrid(np.linspace(-0.9, 0.9, 5), np.linspace(-0.9, 0.9, 5))
zz = np.zeros_like(xx)
planesurf = go.Surface(x=xx, y=yy, z=zz, opacity=1, colorscale=[(0, "rgb(255,255,255)"), (1, "rgb(255,255,255)")])
fig.add_trace(planesurf)
# fig.add_traces(get_surface_line_traces(xx, yy, zz))
fig.show()

## test icp (note doesn't work)

In [ ]:
from sklearn.neighbors import KDTree

def icp_translate(source_pc, target_pc, max_iters=20, tol=1e-3, verbose=False, ntheta=3, nphi=3):
    """source_pc assumed to be smaller than target"""
    src_mean = np.mean(source_pc, axis=0)
    targ_mean = np.mean(target_pc, axis=0)
    scale = np.max(np.linalg.norm(target_pc - targ_mean, axis=1))

    target_kd = KDTree(target_pc)
    thetas = np.linspace(0, 2 * np.pi, ntheta + 1)[:-1]
    phis = np.linspace(0, np.pi, nphi + 2)[1:-1]
    alltheta, allphi = np.meshgrid(thetas, phis)
    alltheta = alltheta.reshape(-1)
    allphi = allphi.reshape(-1)
    offset_choices = scale * np.array([np.sin(allphi) * np.cos(alltheta), np.sin(allphi) * np.sin(alltheta), np.cos(allphi)]).T
    alltranslations = np.zeros((len(alltheta), 3))
    allmeandists = np.zeros(len(alltheta))
    for j, offset in enumerate(offset_choices):
        # p = targ_mean - src_mean
        p = (targ_mean + offset) - src_mean
        prevdist = np.inf
        K = max_iters
        for i in range(K):
            dists, close_idxs = target_kd.query(source_pc + p)
            meandist = np.mean(dists)
            targ_mean_filt = np.mean(target_pc[close_idxs], axis=0)
            p = targ_mean_filt - src_mean
            if np.abs(prevdist - meandist) < tol:
                if verbose:
                    print(f"converged at iter {i}")
                break
            prevdist = meandist
            if i == K - 1:
                if verbose:
                    print(f"max iters {K} reached before tolerance {tol}")
        allmeandists[j] = np.mean(meandist)
        alltranslations[j, :] = p
    bestidx = np.argmin(allmeandists)
    pose = np.eye(4)
    pose[:3, 3] = alltranslations[bestidx]
    return pose

In [ ]:
%timeit T = icp_translate(cylnp, randptssurfpred, max_iters=5, verbose=False, ntheta=3, nphi=2)

In [ ]:
i = 131
cylnp = synthdict["pts"][i].numpy().astype(np.float32)
np.random.seed(5)
cylnp[:, :] += np.random.uniform(-2, 2, 3)
cylpc = v3d.Point3d(p=cylnp)
trialres = trialresults[i]
x1truth, x2truth  = get_cyl_endpoints(trialres["axtruth"], 1, trialres["yoffsettruth"], axidx=1)
x1pred, x2pred  = get_cyl_endpoints(trialres["axpred"], 1, trialres["yshiftpred"], axidx=1)
x, y, z = get_cylinder_surf(x1pred, x2pred, trialres["rpred"])
origcylsurfpred = go.Surface(
    x=x, y=y, z=z,
    colorscale="purples",
    opacity=0.2)
randptssurfpred = random_cylinder_surf(x1truth, x2truth, trialres["rtruth"], 10000)
T = icp_translate(cylnp, randptssurfpred, max_iters=10, verbose=False, ntheta=3, nphi=3)
print(T)
x1pred -= T[:3, 3]
x2pred -= T[:3, 3]
centroid_truth = (x1truth + x2truth) / 2
centroid_pred = (x1pred + x2pred) / 2
truthray = v3d.Ray(pos=centroid_truth, dir=trialres["axtruth"])
predray = v3d.Ray(pos=centroid_pred, dir=trialres["axpred"])
fig = v3d.make_fig([cylpc, predray])
x, y, z = get_cylinder_surf(x1pred, x2pred, trialres["rpred"])
cylsurfpred = go.Surface(
    x=x, y=y, z=z,
    colorscale="oranges",
    opacity=0.2)
fig.add_trace(cylsurfpred)
fig.add_trace(origcylsurfpred)
fig.show()
print(monte_carlo_volume_ratio(10000, x1truth, x2truth, trialres["rtruth"], 0, 1, 0, 0))
print(monte_carlo_volume_ratio(10000, x1pred, x2pred, trialres["rpred"], 0, 1, 0, 0))